# Homework and bakeoff: Multi-domain sentiment

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2023"

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cgpotts/cs224u/blob/main/hw_sentiment.ipynb)
[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/cgpotts/cs224u/blob/main/hw_sentiment.ipynb)

If Colab is opened with this badge, please **save a copy to drive** (from the File menu) before running the notebook.

## Overview

This homework and associated bakeoff are devoted to supervised sentiment analysis in a ternary label setting (positive, negative, neutral). Your ultimate goal is to develop systems that can make accurate predictions in multiple domains.

The homework questions ask you to implement some baseline systems using DynaSent Round 1, DynaSent Round 2, and the Stanford Sentiment Treebank. The bakeoff challenge is to define a system that does well on the DynaSent test sets, the SST-3 test set, and a set of mystery examples that don't correspond to the DynaSent or SST-3 domains.

__Important methodological note:__ The DynaSent and SST-3 test sets are already publicly distributed, so we are counting on people not to cheat by developing their models on these test sets. You must do all your development without using these test sets at all, and then evaluate exactly once on the test set and turn in the results, with no further system tuning or additional runs. _Much of the scientific integrity of our field depends on people adhering to this honor code._

This notebook briefly introduces our three development datasets, states the homework questions, and then provides guidance on the original system and associated bakeoff entry.

## Set-up

In [2]:
!git clone https://github.com/cgpotts/cs224u.git
import sys
sys.path.append("cs224u")

Cloning into 'cs224u'...
remote: Enumerating objects: 2239, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 2239 (delta 68), reused 110 (delta 60), pack-reused 2092
Receiving objects: 100% (2239/2239), 41.50 MiB | 18.95 MiB/s, done.
Resolving deltas: 100% (1367/1367), done.


In [3]:
from collections import defaultdict, Counter
from datasets import load_dataset
import pandas as pd
from sklearn.metrics import classification_report
import torch
import torch.nn as nn

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Datasets

### DynaSent round 1

The DynaSent dataset of [Potts, Wu, et al. 2021](https://aclanthology.org/2021.acl-long.186/) is a ternary sentiment benchmark consisting of two rounds (so far). The dataset is available on [Hugging Face](https://huggingface.co/datasets/dynabench/dynasent).

For Round 1, the authors collected sentences from the [Yelp Academic Dataset](https://www.yelp.com/dataset) that fooled a top-performing sentiment model but were intuitive for humans. The model was used only to heuristically find the examples. Crowdworkers multiply-labeled all of them.

The round contains a lot of metadata that could be useful for developing sentiment models. We will focus on just the sentences and labels, but you are free to make use of this additional metadata in developing uour system.

In [4]:
dynasent_r1 = load_dataset("dynabench/dynasent", 'dynabench.dynasent.r1.all')

Generating train split:   0%|          | 0/80488 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3600 [00:00<?, ? examples/s]

Dataset dynabench_dyna_sent downloaded and prepared to /root/.cache/huggingface/datasets/dynabench___dynabench_dyna_sent/dynabench.dynasent.r1.all/1.1.0/ab89971d9ae1aacc59ed44d6855bf0e89167417257e2c2666f38e532148f2967. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
dynasent_r1

DatasetDict({
    train: Dataset({
        features: ['id', 'hit_ids', 'sentence', 'indices_into_review_text', 'model_0_label', 'model_0_probs', 'text_id', 'review_id', 'review_rating', 'label_distribution', 'gold_label', 'metadata'],
        num_rows: 80488
    })
    validation: Dataset({
        features: ['id', 'hit_ids', 'sentence', 'indices_into_review_text', 'model_0_label', 'model_0_probs', 'text_id', 'review_id', 'review_rating', 'label_distribution', 'gold_label', 'metadata'],
        num_rows: 3600
    })
    test: Dataset({
        features: ['id', 'hit_ids', 'sentence', 'indices_into_review_text', 'model_0_label', 'model_0_probs', 'text_id', 'review_id', 'review_rating', 'label_distribution', 'gold_label', 'metadata'],
        num_rows: 3600
    })
})

Splits:

In [6]:
def print_label_dist(dataset, labelname='gold_label', splitnames=('train', 'validation')):
    for splitname in splitnames:
        print(splitname)
        dist = sorted(Counter(dataset[splitname][labelname]).items())
        for k, v in dist:
            print(f"\t{k:>14s}: {v}")

In [7]:
print_label_dist(dynasent_r1)

train
	      negative: 14021
	       neutral: 45076
	      positive: 21391
validation
	      negative: 1200
	       neutral: 1200
	      positive: 1200


### DynaSent round 2

DynaSent Round 2 was created using different methods than Round 1. For Round 2, crowdworkers edited sentences from the Yelp Academic Dataset seeking to achieve a particular sentiment goal (e.g., expressing a positive sentiment) while fooling a top-performing model. This work was done on the [Dynabench](https://dynabench.org) platform. The hope is that this directly adversarial goal will lead to examples that are very hard for present-day models but intuitive for humans. All the examples were multiply-labeled by separate annotators.

In [8]:
dynasent_r2 = load_dataset("dynabench/dynasent", 'dynabench.dynasent.r2.all')

Generating train split:   0%|          | 0/13065 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/720 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/720 [00:00<?, ? examples/s]

Dataset dynabench_dyna_sent downloaded and prepared to /root/.cache/huggingface/datasets/dynabench___dynabench_dyna_sent/dynabench.dynasent.r2.all/1.1.0/ab89971d9ae1aacc59ed44d6855bf0e89167417257e2c2666f38e532148f2967. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
print_label_dist(dynasent_r2)

train
	      negative: 4579
	       neutral: 2448
	      positive: 6038
validation
	      negative: 240
	       neutral: 240
	      positive: 240


### Stanford Sentiment Treebank

The [Stanford Sentiment Treebank (SST)](http://nlp.stanford.edu/sentiment/) of [Socher et al. 2013](https://aclanthology.org/D13-1170/) is a widely-used resource for evaluating supervised models. It consists of sentences from Rotten Tomatoes Movie Reviews (see [Pang and Lee's project page](https://www.cs.cornell.edu/home/llee/papers/pang-lee-stars.home.html)). We will use the ternary version of the task (SST-3).

SST examples are special in that they are labeled at the phrase-level as well as the sentence level, which provides very extensive and detailed supervision for sentiment. We will use only the sentence-level labels for the homework, but you are free to use the phrase-level labels as well in designing your original system. (To do this, you will need to get the dataset from the above project page, since the Hugging Face SST-3 we are using does not include these labels.)

In [10]:
sst = load_dataset("SetFit/sst5")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/SetFit--sst5-215b01185f4be506/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
sst

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 8544
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2210
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1101
    })
})

Out of the box, this is a five-way task:

In [12]:
print_label_dist(sst, labelname='label_text')

train
	      negative: 2218
	       neutral: 1624
	      positive: 2322
	 very negative: 1092
	 very positive: 1288
validation
	      negative: 289
	       neutral: 229
	      positive: 279
	 very negative: 139
	 very positive: 165


The above labels are not aligned with our ternary task, and the dataset distribution uses slightly different keys from those of DynaSent. The following code converts the dataset to SST-3 and also aligns the dataset keys:

In [13]:
def convert_sst_label(s):
    return s.split(" ")[-1]

In [14]:
for splitname in ('train', 'validation', 'test'):
    dist = [convert_sst_label(s) for s in sst[splitname]['label_text']]
    sst[splitname] = sst[splitname].add_column('gold_label', dist)
    sst[splitname] = sst[splitname].add_column('sentence', sst[splitname]['text'])

In [15]:
print_label_dist(sst)

train
	      negative: 3310
	       neutral: 1624
	      positive: 3610
validation
	      negative: 428
	       neutral: 229
	      positive: 444


## Question 2: Transformer fine-tuning

We're now going to move into a more modern mode: fine-tuning pretrained components.

We'll use BERT-mini (originally from [the BERT repo](https://github.com/google-research/bert)) for the homework so that we can rapdily develop prototypes. You can then consider scaling up to larger models.

In [16]:
import transformers
from transformers import AutoModel, AutoTokenizer

The `transformers` library does a lot of logging. To avoid ending up with a cluttered notebook, I am changing the logging level. You might want to skip this as you scale up to building production systems, since the logging is very good – it gives you a lot of insights into what the models and code are doing.

In [17]:
transformers.logging.set_verbosity_error()

Here we set ourselves up to use BERT-mini:

In [18]:
weights_name = "prajjwal1/bert-mini"

bert = AutoModel.from_pretrained(weights_name)

bert_tokenizer = AutoTokenizer.from_pretrained(weights_name)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

### Background: Tokenization

Tokenization in Transformer models is handled differently from tokenization in linear models of the sort we used in Question 1. For Transformer models, we need to use the tokenizer that comes with the model so that we reliably have embedding representations for every token.

In [19]:
example_text = "Bert knows Snuffleupagus"

Here's a basic tokenization step:

In [20]:
bert_tokenizer.tokenize(example_text)

['bert', 'knows', 's', '##nu', '##ffle', '##up', '##ag', '##us']

Notice that the tokenizer split "Snuffleupagus" into a bunch of subword tokens.

The above use of the tokenizer, where we map from strings to lists of strings, is really for us humans. For modeling, the most important step for tokenization is mapping individual strings to sequences of integer ids. These ids key into the lowest embedding layer of the model.

In [21]:
ex_ids = bert_tokenizer.encode(example_text, add_special_tokens=True)

ex_ids

[101, 14324, 4282, 1055, 11231, 18142, 6279, 8490, 2271, 102]

We can get map these indices back to "words" if we want:

In [22]:
bert_tokenizer.convert_ids_to_tokens(ex_ids)

['[CLS]',
 'bert',
 'knows',
 's',
 '##nu',
 '##ffle',
 '##up',
 '##ag',
 '##us',
 '[SEP]']

### Background: Representation

Having mapped our string to a list of tokens, we can use the `forward` method of the model to get representations:

In [23]:
with torch.no_grad():
    reps = bert(torch.tensor([ex_ids]))

There are a lot of options for which representations to get. With the above call, we got the following:

In [24]:
reps.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

The value of `last_hidden_state` hidden state is the sequence of final output states from the model:

In [25]:
reps.last_hidden_state.shape

torch.Size([1, 10, 256])

This is: 1 example, 10 token representations, each one a 256 dimension vector.

The value of `pooler_output` is a set of currently random parameters sitting on top of the first output hidden state. You can see here that it is a single vector representation per example:

In [26]:
reps.pooler_output.shape

torch.Size([1, 256])

I often feel unsure of precisely what this model component is. Here we can have a quick look:

In [27]:
bert.pooler

BertPooler(
  (dense): Linear(in_features=256, out_features=256, bias=True)
  (activation): Tanh()
)

So this is a dense linear layer (a single matrix of weights) with a bias term, and a tanh activation function is applied to the output. We could put a classifier head on top of this if we wanted to, but we might have mixed feelings about being stuck with that tanh step.

### Background: Masking

Where examples from a single batch have different lengths, we need to mask the padded tokens to get the intended results from the model.

For a quick example, here we process our full example from above and print out the first five values:

In [28]:
with torch.no_grad():
    reps = bert(torch.tensor([ex_ids]))
    print(reps.last_hidden_state[0][0][: 5])

tensor([-0.3763, -0.3209,  0.8817,  0.4568, -1.0314])


And now we do the same thing, but with masking of the final five positions to illustate:

In [29]:
with torch.no_grad():
    # Mask the last 5 tokens:
    am = torch.tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]])
    maskreps = bert(torch.tensor([ex_ids]), attention_mask=am)
    print(maskreps.last_hidden_state[0][0][: 5])

tensor([-0.1793, -0.8994,  0.9695,  0.9130, -0.7129])


### Task 1: Batch tokenization [1 point]

Your task here is to use the `batch_encode_plus` method for `bert_tokenizer` to tokenize a list of strings. You should complete `get_batch_token_ids` according to the specification in the doctring. All these steps can be handled with a single call to `batch_encode_plus`.

In [30]:
def get_batch_token_ids(batch, tokenizer):
    """Map `batch` to a tensor of ids. The return
    value should meet the following specification:

    1. The max length should be 512.
    2. Examples longer than the max length should be truncated
    3. Examples should be padded to the max length for the batch.
    4. The special [CLS] should be added to the start and the special
       token [SEP] should be added to the end.
    5. The attention mask should be returned
    6. The return value of each component should be a tensor.

    Parameters
    ----------
    batch: list of str
    tokenizer: Hugging Face tokenizer

    Returns
    -------
    dict with at least "input_ids" and "attention_mask" as keys,
    each with Tensor values

    """
    return tokenizer(batch, max_length=512, padding=True, truncation=True, add_special_tokens=True, return_tensors="pt")



Here's a test you can use:

In [31]:
def test_get_batch_token_ids(func):
    examples = [
        "Bert knows Snuffleupagus",
        "ELMo knew Bert.",
        "Buffalo " * 520
    ]
    test_tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-mini")
    result = func(examples, test_tokenizer)
    errcount = 0
    if 'attention_mask' not in result:
        errcount += 1
        print(f"Error for `{func.__name__}`: "
              f"Attention mask was not returned")
    ids = result['input_ids']
    if not isinstance(ids, torch.Tensor):
        errcount += 1
        print(f"Error for `{func.__name__}`: "
              f"Return values are not tensors")
    if ids.shape[1] != 512:
        errcount += 1
        print(f"Error for `{func.__name__}`: "
              f"Expected sequence length 512; got {ids.shape[1]}")
    if ids[0][0] != bert_tokenizer.cls_token_id:
        errcount += 1
        print(f"Error for `{func.__name__}`: "
              f"Special tokens were not added")
    if errcount == 0:
        print(f"No errors found for `{func.__name__}`")

In [32]:
test_get_batch_token_ids(get_batch_token_ids)

No errors found for `get_batch_token_ids`


### Task 2: Contextual representations [1 point]

This next task is not used directly in fine-tuning, but it should help ensure that you understand how BERT representations are created and how they need to be managed.

Your task is to complete `get_reps` so that, given a dataset (list of strings), it returns a single tensor in which each row is the output hidden state above the [CLS] token for that example. `gets_reps` has a batchsize argument that the user can manage depending on how much available memory they have and how large their model is.

In [33]:
def get_reps(dataset, model, tokenizer, batchsize=20):
    """Represent each example in `dataset` with the final hidden state
    above the [CLS] token.

    Parameters
    ----------
    dataset : list of str
    model : BertModel
    tokenizer : BertTokenizerFast
    batchsize : int

    Returns
    -------
    torch.Tensor with shape `(n_examples, dim)` where `dim` is the
    dimensionality of the representations for `model`

    """
    data = []
    with torch.no_grad():
        # Iterate over `dataset` in batches:
        for i in range(0, len(dataset), batchsize):



            # Encode the batch with `get_batch_token_ids`:
            batch_sentence = get_batch_token_ids(dataset[i:i + batchsize], tokenizer)



            # Get the representations from the model, making
            # sure to pay attention to masking:
            sentence_rep = model(batch_sentence["input_ids"], attention_mask=batch_sentence["attention_mask"])

            data.append(sentence_rep.last_hidden_state[:, 0, :])

        # Return a single tensor:
        return torch.cat(data)




Quick test:

In [34]:
def test_get_reps(func):
    examples = ["The cat slept.", "The bird chirped."] * 20
    weights_name = "prajjwal1/bert-mini"
    test_model = AutoModel.from_pretrained(weights_name)
    test_tokenizer = AutoTokenizer.from_pretrained(weights_name)
    result = func(examples, test_model, test_tokenizer, batchsize=2)

    errcount = 0
    if result.shape != (40, 256):
        print(f"Error for `{func.__name__}`: "
              f"Expected shape {(40, 256)}, got {result.shape}")
    if round(result[0][0].item(), 2) != -0.64:
        print(f"Error for `{func.__name__}`: "
              f"Representations seem to be incorrect")
    print(f"No errors found for `{func.__name__}`")

In [35]:
test_get_reps(get_reps)

No errors found for `get_reps`


### Task 3: Fine-tuning module [1 point]

In [36]:
class BertClassifierModule(nn.Module):
    def __init__(self,
            n_classes,
            hidden_activation,
            weights_name="prajjwal1/bert-mini"):
        """This module loads a Transformer based on  `weights_name`,
        puts it in train mode, add a dense layer with activation
        function give by `hidden_activation`, and puts a classifier
        layer on top of that as the final output. The output of
        the dense layer should have the same dimensionality as the
        model input.

        Parameters
        ----------
        n_classes : int
            Number of classes for the output layer
        hidden_activation : torch activation function
            e.g., nn.Tanh()
        weights_name : str
            Name of pretrained model to load from Hugging Face

        """
        super().__init__()
        self.n_classes = n_classes
        self.weights_name = weights_name
        self.bert = AutoModel.from_pretrained(self.weights_name)
        self.bert.train()
        self.hidden_activation = hidden_activation
        self.hidden_dim = self.bert.embeddings.word_embeddings.embedding_dim
        # Add the new parameters here using `nn.Sequential`.
        # We can define this layer as
        #
        #  h = f(cW1 + b_h)
        #  y = hW2 + b_y
        #
        # where c is the final hidden state above the [CLS] token,
        # W1 has dimensionality (self.hidden_dim, self.hidden_dim),
        # W2 has dimensionality (self.hidden_dim, self.n_classes),
        # f is the hidden activation, and we rely on the PyTorch loss
        # function to add apply a softmax to y.
        self.classifier_layer = nn.Sequential(
            nn.Linear(self.hidden_dim, self.hidden_dim),
            self.hidden_activation,
            nn.Linear(self.hidden_dim, self.n_classes),
        )



    def forward(self, indices, mask):
        """Process `indices` with `mask` by feeding these arguments
        to `self.bert` and then feeding the initial hidden state
        in `last_hidden_state` to `self.classifier_layer`

        Parameters
        ----------
        indices : tensor.LongTensor of shape (n_batch, k)
            Indices into the `self.bert` embedding layer. `n_batch` is
            the number of examples and `k` is the sequence length for
            this batch
        mask : tensor.LongTensor of shape (n_batch, d)
            Binary vector indicating which values should be masked.
            `n_batch` is the number of examples and `k` is the
            sequence length for this batch

        Returns
        -------
        tensor.FloatTensor
            Predicted values, shape `(n_batch, self.n_classes)`

        """
        feature_rep = self.bert(indices, attention_mask=mask)
        return self.classifier_layer(feature_rep.last_hidden_state[:, 0, :])

In [37]:
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier

class BertClassifier(TorchShallowNeuralClassifier):
    def __init__(self, weights_name, *args, **kwargs):
        self.weights_name = weights_name
        self.tokenizer = AutoTokenizer.from_pretrained(self.weights_name)
        super().__init__(*args, **kwargs)
        self.params += ['weights_name']

    def build_graph(self):
        return BertClassifierModule(
            self.n_classes_, self.hidden_activation, self.weights_name)

    def build_dataset(self, X, y=None):
        data = get_batch_token_ids(X, self.tokenizer)
        if y is None:
            dataset = torch.utils.data.TensorDataset(
                data['input_ids'], data['attention_mask'])
        else:
            self.classes_ = sorted(set(y))
            self.n_classes_ = len(self.classes_)
            class2index = dict(zip(self.classes_, range(self.n_classes_)))
            y = [class2index[label] for label in y]
            y = torch.tensor(y)
            dataset = torch.utils.data.TensorDataset(
                data['input_ids'], data['attention_mask'], y)
        return dataset

And here is a training run that should do pretty well for our problem.

__Note__: This step should not be run on CPU machines. On Google Colab with a GPU, it will likely take about an hour.

In [38]:
bert_finetune = BertClassifier(
    weights_name="prajjwal1/bert-medium",
    hidden_activation=nn.ReLU(),
    eta=0.00005,          # Low learning rate for effective fine-tuning.
    batch_size=8,         # Small batches to avoid memory overload.
    gradient_accumulation_steps=4,  # Increase the effective batch size to 32.
    early_stopping=True,  # Early-stopping
    n_iter_no_change=5)   # params.

In [39]:
%%time

_ = bert_finetune.fit(
    dynasent_r1['train']['sentence'],
    dynasent_r1['train']['gold_label'])

Stopping after epoch 9. Validation score did not improve by tol=1e-05 for more than 5 epochs. Final error is 227.83749515352247

CPU times: user 3h 31min 43s, sys: 17.7 s, total: 3h 32min 1s
Wall time: 3h 32min 59s


In [40]:
preds = bert_finetune.predict(sst['validation']['sentence'])

In [41]:
print(classification_report(sst['validation']['gold_label'], preds, digits=3))

              precision    recall  f1-score   support

    negative      0.592     0.610     0.601       428
     neutral      0.300     0.380     0.335       229
    positive      0.695     0.579     0.631       444

    accuracy                          0.550      1101
   macro avg      0.529     0.523     0.522      1101
weighted avg      0.573     0.550     0.558      1101



In [42]:
preds = bert_finetune.predict(dynasent_r1['validation']['sentence'])

In [43]:
print(classification_report(dynasent_r1['validation']['gold_label'], preds, digits=3))

              precision    recall  f1-score   support

    negative      0.815     0.645     0.720      1200
     neutral      0.645     0.875     0.743      1200
    positive      0.800     0.682     0.736      1200

    accuracy                          0.734      3600
   macro avg      0.753     0.734     0.733      3600
weighted avg      0.753     0.734     0.733      3600

